In [ ]:
# setup
import time, os, calendar, sys
import envkey
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
%matplotlib inline
from pandas.plotting import register_matplotlib_converters
import seaborn as sns
from sqlalchemy.engine import create_engine
import math
import datetime

# allow importing modules from ../..
sys.path.insert(1, os.path.join(sys.path[0], '../..'))

register_matplotlib_converters()
sns.set(rc={'figure.figsize':(11, 4)})

ANALYTICS_MYSQL_ENDPOINT = os.getenv('ANALYTICS_MYSQL_ENDPOINT')
ANALYTICS_MYSQL_PASSWORD = os.getenv('ANALYTICS_MYSQL_PASSWORD')

if not (ANALYTICS_MYSQL_ENDPOINT and ANALYTICS_MYSQL_PASSWORD):
    raise Exception('ANALYTICS_MYSQL credentials not found')

def get_engine(game):
    database_name = '%s_upcache' % game
    return create_engine(f'mysql://analytics1:{ANALYTICS_MYSQL_PASSWORD}@{ANALYTICS_MYSQL_ENDPOINT}/{database_name}', connect_args={'connect_timeout': 10})

def get_quarter_time_boundaries(timestamp):
    q = {}
    q['start'] = 1648080000
    q['end'] = q['start'] + (86400 * 7 * 14) # 14 weeks, 13 weeks for the quarter, plus one trailing week
    while not (timestamp >= q['start'] and timestamp <= q['end']):
        q['start'] = q['end']
        q['end'] = q['start'] + (86400 * 7 * 14)
    return q

def get_growth_okr_dau_sql(game,start_time,end_time,frame_platform=None):
    frame_platform_sql = ''
    if frame_platform:
        frame_platform_sql = "AND acq.frame_platform = '%s'" % frame_platform
    return """
    SELECT master.day AS `day`,
    (SELECT ROUND(SUM(dau)/3)
    FROM %s_upcache.%s_sessions_daily_summary AS acq
    WHERE acq.day >= master.day - 3*86400
    AND acq.day < master.day
    %s
    AND acq.country_tier IN ('1','2')) AS `dau`
    FROM skynet.bh_daily_summary master
    WHERE master.day >= %d - 90*86400
    AND master.day < %d
    GROUP BY master.day;
    """ % (game, game, frame_platform_sql,start_time,end_time)

print(f"Dashboard updated %s" % time.strftime('%a, %d %b %Y at %H:%M:%S UTC', time.gmtime()))
time_now = int(time.time())
quarter_boundaries = get_quarter_time_boundaries(time_now)
start_time = quarter_boundaries['start']
end_time = quarter_boundaries['end']

# prepare growth OKR data
game_frames = {'tr':['fb','bh','k2'],'fs':['bh']}
growth_okr_dau = {}
engine = get_engine('tr')
for game in ['tr','dv','mf2','bfm','mf','fs']:
    if game in game_frames:
        for frame in game_frames[game]:
            dau_sql = get_growth_okr_dau_sql(game,start_time,end_time,frame)
            growth_okr_dau_data = pd.read_sql(dau_sql, engine)
            if 'day' not in growth_okr_dau:
                growth_okr_dau['day'] = []
                for day in growth_okr_dau_data['day']:
                    growth_okr_dau['day'].append(day)
            key = game.upper() + ' (' +  frame.upper() + ')'
            growth_okr_dau[key] = []
            for day in growth_okr_dau_data['dau']:
                growth_okr_dau[key].append(day)
            if 'total' not in growth_okr_dau:
                growth_okr_dau['total'] = []
                for day in growth_okr_dau_data['dau']:
                    growth_okr_dau['total'].append(day)
            else:
                for i, day in enumerate(growth_okr_dau_data['dau']):
                    growth_okr_dau['total'][i] += day
    else:
        dau_sql = get_growth_okr_dau_sql(game,start_time,end_time)
        growth_okr_dau_data = pd.read_sql(dau_sql, engine)
        if 'day' not in growth_okr_dau:
            dau['day'] = []
            for day in growth_okr_dau_data['day']:
                dau['day'].append(day)
        key = game.upper()
        growth_okr_dau[key] = []
        for day in growth_okr_dau_data['dau']:
            growth_okr_dau[key].append(day)
        if 'total' not in growth_okr_dau:
            dau['total'] = []
            for day in growth_okr_dau_data['dau']:
                growth_okr_dau['total'].append(day)
        else:
            for i, day in enumerate(growth_okr_dau_data['dau']):
                growth_okr_dau['total'][i] += day

Dashboard updated Sat, 29 Jan 2022 at 02:47:22 UTC


In [ ]:
# Growth OKR display
print('All Game Tier 1/2 DAU, average of trailing 3 days')
growth_okr_boundaries = get_quarter_time_boundaries(time_now)
growth_okr_boundaries['start'] = growth_okr_boundaries['start'] + 86400*7
for i, day in enumerate(growth_okr_dau['day']):
    if day < growth_okr_boundaries['start'] or day > growth_okr_boundaries['end']: continue
    if datetime.datetime.utcfromtimestamp(day).strftime('%A') == 'Thursday':
        print(datetime.datetime.utcfromtimestamp(day).strftime('%Y-%m-%d') + ': ' + str(int(growth_okr_dau['total'][i])))
print('Change since last week (if any)')
game_frames = {'tr':['fb','bh','k2'],'fs':['bh']}
for i, day in enumerate(growth_okr_dau['day']):
    if day < growth_okr_boundaries['start'] or day > growth_okr_boundaries['end']: continue
    if datetime.datetime.utcfromtimestamp(day).strftime('%A') == 'Thursday':
        output_line = datetime.datetime.utcfromtimestamp(day).strftime('%Y-%m-%d') + ': '
        total_changes = 0
        for game in ['tr','dv','mf2','bfm','mf','fs']:
            if game in game_frames:
                for frame in game_frames[game]:
                    key = game.upper() + ' (' +  frame.upper() + ')'
                    this_week = int(growth_okr_dau[key][i])
                    last_week = int(growth_okr_dau[key][i-1])
                    change = this_week - last_week
                    if change != 0:
                        if change > 0:
                            output_line += ' %s: +%d,' % (key.replace('K2','KG'), change)
                        else:
                            output_line += ' %s: %d,' % (key.replace('K2','KG'), change)
                        total_changes += 1
            else:
                key = game.upper()
                this_week = growth_okr_dau[key][i]
                last_week = growth_okr_dau[key][i-1]
                change = this_week - last_week
                if change != 0:
                    if change > 0:
                        output_line += ' %s: +%d,' % (key.replace('K2','KG'), change)
                    else:
                        output_line += ' %s: %d,' % (key.replace('K2','KG'), change)
                    total_changes += 1
        if total_changes == 0:
            output_line += 'No Change'
        print(output_line)